In [1]:
import csv
import pandas as pd
import numpy as np


In [1]:
seebase = pd.read_csv ('Seebase_data.csv')
wells = pd.read_csv ('TE_data.csv')
top_basement = pd.read_csv ('Top_basement_MD.csv')

#convert top_basement df into dictionary
top_basement_dict=top_basement.set_index('well_name')['top_basem'].to_dict()

#Map the dataframe column values with the Dictionary key 
wells['top_basement_MD']=wells['well_name'].map(top_basement_dict)

#convert seebase df into dictionary
seebase_dict=seebase.set_index('well_seebase')['seebase_MD'].to_dict()

#Map the dataframe column values with the Dictionary key 
wells['top_seebase_MD']=wells['well_name'].map(seebase_dict)


In [ ]:
wells.head()

In [ ]:
wells_small=wells.loc[:,['well_name','zone','gross_thickness']]
print(wells_small)

In [ ]:
#pivot rows to columns with multiple values
wells_small_reshaped = wells_small.pivot_table(index=['well_name'], columns='zone', values=['gross_thickness'])

# flatten columns multi-index
wells_small_reshaped.columns = ['_'.join(col).strip() for col in wells_small_reshaped.columns.values]
# bring `well-name` as column
wells_small_reshaped = wells_small_reshaped.reset_index()
wells_small_reshaped

In [ ]:
#drop unnecessary columns from wells dataframe before merging 
wells_new = wells.drop(['zone','gross_thickness'], 1)

#merge 2 dataframes based on the column well_name
final=pd.merge(wells_small_reshaped, wells_new, how='inner', on=['well_name'])

final = final.drop_duplicates('well_name', keep='last')     #drop duplicates
final.reset_index(drop=True, inplace=True)          #reset index
final

In [6]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   well_name               624 non-null    object 
 1   gross_thickness_Augila  432 non-null    float64
 2   gross_thickness_Gialo   547 non-null    float64
 3   gross_thickness_Gir     599 non-null    float64
 4   gross_thickness_Zelten  312 non-null    float64
 5   lat                     624 non-null    float64
 6   long                    624 non-null    float64
 7   top_basement_MD         624 non-null    float64
 8   top_seebase_MD          624 non-null    int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 44.0+ KB


In [7]:
final.describe()

,gross_thickness_Augila,gross_thickness_Gialo,gross_thickness_Gir,gross_thickness_Zelten,lat,long,top_basement_MD,top_seebase_MD
count,432.000000,547.000000,599.000000,312.000000,624.000000,624.000000,624.000000,624.000000
mean,85.669925,396.410883,517.711349,135.400321,28.254214,20.824255,2671.255721,2776.538462
std,34.740303,102.376453,99.622397,73.135630,0.545513,1.333332,679.339969,773.019666
min,18.600000,6.700000,64.400000,0.300000,26.103963,17.179581,690.410000,426.000000
25%,64.600000,337.400000,478.800000,87.350000,28.017987,19.900713,2055.907500,2088.500000
50%,81.800000,383.700000,511.100000,146.600000,28.214547,21.180901,2757.870000,2877.500000
75%,103.625000,452.600000,559.400000,189.525000,28.478456,21.902258,3122.177500,3141.500000
max,242.300000,659.900000,790.300000,481.900000,29.791266,24.247500,4872.210000,6197.000000


In [ ]:
#final_dropna=final.dropna()     #drop nan values
#final_dropna.reset_index(drop=True, inplace=True)          #reset index
#final_dropna


# Replace a value in df with NaN
# df.replace(to_replace=-9999, value=np.nan)

In [ ]:
#fill NaN with AVG values
final.mean()
final_avg=final.fillna(final.mean())
final_avg

In [9]:

Q1 = final_avg.quantile(0.20)

Q3 = final_avg.quantile(0.80)

IQR = Q3 - Q1



final_avg_clean = final_avg[~((final_avg < (Q1 - 1.5 * IQR)) |(final_avg > (Q3 + 1.5 * IQR))).any(axis=1)]

final_avg_clean.describe()

<ipython-input-9-a47c7c9d5729>:9: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  final_avg_clean = final_avg[~((final_avg < (Q1 - 1.5 * IQR)) |(final_avg > (Q3 + 1.5 * IQR))).any(axis=1)]
<ipython-input-9-a47c7c9d5729>:9: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  final_avg_clean = final_avg[~((final_avg < (Q1 - 1.5 * IQR)) |(final_avg > (Q3 + 1.5 * IQR))).any(axis=1)]


,gross_thickness_Augila,gross_thickness_Gialo,gross_thickness_Gir,gross_thickness_Zelten,lat,long,top_basement_MD,top_seebase_MD
count,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000
mean,84.460907,401.299365,515.870090,143.479598,28.241113,20.856814,2653.436819,2720.619543
std,21.537534,78.337372,74.038344,28.140894,0.460262,1.207743,635.426353,659.157400
min,22.900000,169.200000,307.800000,69.500000,27.022159,17.179581,1506.610000,1531.000000
25%,77.400000,353.600000,480.700000,135.400321,28.008980,19.908088,2041.600000,2069.000000
50%,85.669925,396.410883,511.100000,135.400321,28.202871,21.206680,2770.620000,2880.000000
75%,92.300000,437.400000,549.600000,150.000000,28.395244,21.849683,3069.300000,3115.000000
max,147.200000,643.200000,729.100000,208.200000,29.572467,23.676663,4872.210000,4980.000000


In [10]:
#Save df to csv

final_avg.to_csv(r'\\main.glb.corp.local\ep-hq$\Home\DEF\4\J1040904\Desktop\Projects\Data science\Basement project\Python\final_data.csv', index = False)

final_avg_clean.to_csv(r'\\main.glb.corp.local\ep-hq$\Home\DEF\4\J1040904\Desktop\Projects\Data science\Basement project\Python\final_data_clean.csv', index = False)